In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib

In [2]:
spark = SparkSession\
        .builder\
        .getOrCreate()

22/06/01 21:41:26 WARN Utils: Your hostname, razevedo resolves to a loopback address: 127.0.1.1; using 192.168.1.86 instead (on interface enp8s0)
22/06/01 21:41:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/01 21:41:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load the dataset

In [3]:
df_log = spark.read.text("data/NASA_access_log_Jul95.gz")

In [4]:
df_log.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
df_log.count()

1891715

In [6]:
df_log.show(10)

+--------------------+
|               value|
+--------------------+
|199.72.81.55 - - ...|
|unicomp6.unicomp....|
|199.120.110.21 - ...|
|burger.letters.co...|
|199.120.110.21 - ...|
|burger.letters.co...|
|burger.letters.co...|
|205.212.115.106 -...|
|d104.aa.net - - [...|
|129.94.144.152 - ...|
+--------------------+
only showing top 10 rows



In [7]:
df_log.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
|burger.letters.com - - 

In [8]:
# parsing
from pyspark.sql.functions import split

new_df = df_log.withColumn("tokenized", split("value", " "))
new_df.limit(10).toPandas()

,value,tokenized
0,199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ...,"[199.72.81.55, -, -, [01/Jul/1995:00:00:01, -0..."
1,unicomp6.unicomp.net - - [01/Jul/1995:00:00:06...,"[unicomp6.unicomp.net, -, -, [01/Jul/1995:00:0..."
2,199.120.110.21 - - [01/Jul/1995:00:00:09 -0400...,"[199.120.110.21, -, -, [01/Jul/1995:00:00:09, ..."
3,burger.letters.com - - [01/Jul/1995:00:00:11 -...,"[burger.letters.com, -, -, [01/Jul/1995:00:00:..."
4,199.120.110.21 - - [01/Jul/1995:00:00:11 -0400...,"[199.120.110.21, -, -, [01/Jul/1995:00:00:11, ..."
5,burger.letters.com - - [01/Jul/1995:00:00:12 -...,"[burger.letters.com, -, -, [01/Jul/1995:00:00:..."
6,burger.letters.com - - [01/Jul/1995:00:00:12 -...,"[burger.letters.com, -, -, [01/Jul/1995:00:00:..."
7,205.212.115.106 - - [01/Jul/1995:00:00:12 -040...,"[205.212.115.106, -, -, [01/Jul/1995:00:00:12,..."
8,"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""...","[d104.aa.net, -, -, [01/Jul/1995:00:00:13, -04..."
9,129.94.144.152 - - [01/Jul/1995:00:00:13 -0400...,"[129.94.144.152, -, -, [01/Jul/1995:00:00:13, ..."


In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

@udf(MapType(StringType(),StringType()))
def parseUDFbetter(line):
    import re
    PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'
    match = re.search(PATTERN, line)
    if match is None:
        return (line, 0)
    size_field = match.group(9)
    if size_field == '-':
        size = 0
    else:
        size = match.group(9)
    return {
        "host"          : match.group(1), 
        "client_identd" : match.group(2), 
        "user_id"       : match.group(3), 
        "date_time"     : match.group(4), 
        "method"        : match.group(5),
        "endpoint"      : match.group(6),
        "protocol"      : match.group(7),
        "response_code" : int(match.group(8)),
        "content_size"  : size
    }

In [10]:
df_parsed = df_log.withColumn("parsed", parseUDFbetter("value"))
df_parsed.limit(10).toPandas()

,value,parsed
0,199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
1,unicomp6.unicomp.net - - [01/Jul/1995:00:00:06...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
2,199.120.110.21 - - [01/Jul/1995:00:00:09 -0400...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
3,burger.letters.com - - [01/Jul/1995:00:00:11 -...,"{'response_code': '304', 'protocol': 'HTTP/1.0..."
4,199.120.110.21 - - [01/Jul/1995:00:00:11 -0400...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
5,burger.letters.com - - [01/Jul/1995:00:00:12 -...,"{'response_code': '304', 'protocol': 'HTTP/1.0..."
6,burger.letters.com - - [01/Jul/1995:00:00:12 -...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
7,205.212.115.106 - - [01/Jul/1995:00:00:12 -040...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
8,"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""...","{'response_code': '200', 'protocol': 'HTTP/1.0..."
9,129.94.144.152 - - [01/Jul/1995:00:00:13 -0400...,"{'response_code': '200', 'protocol': 'HTTP/1.0..."


In [11]:
df_parsed.printSchema()

root
 |-- value: string (nullable = true)
 |-- parsed: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [12]:
df_parsed.select("parsed").limit(10).toPandas()

,parsed
0,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
1,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
2,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
3,"{'response_code': '304', 'protocol': 'HTTP/1.0..."
4,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
5,"{'response_code': '304', 'protocol': 'HTTP/1.0..."
6,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
7,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
8,"{'response_code': '200', 'protocol': 'HTTP/1.0..."
9,"{'response_code': '200', 'protocol': 'HTTP/1.0..."


In [13]:
fields = ["host", "client_identd","user_id", "date_time", "method", "endpoint", "protocol", "response_code", "content_size"]
exprs = ["parsed['{}'] as {}".format(field,field) for field in fields]
exprs

["parsed['host'] as host",
 "parsed['client_identd'] as client_identd",
 "parsed['user_id'] as user_id",
 "parsed['date_time'] as date_time",
 "parsed['method'] as method",
 "parsed['endpoint'] as endpoint",
 "parsed['protocol'] as protocol",
 "parsed['response_code'] as response_code",
 "parsed['content_size'] as content_size"]

In [14]:
df_clean = df_parsed.selectExpr(*exprs)
df_clean.limit(5).toPandas()

,host,client_identd,user_id,date_time,method,endpoint,protocol,response_code,content_size
0,199.72.81.55,-,-,01/Jul/1995:00:00:01 -0400,GET,/history/apollo/,HTTP/1.0,200,6245
1,unicomp6.unicomp.net,-,-,01/Jul/1995:00:00:06 -0400,GET,/shuttle/countdown/,HTTP/1.0,200,3985
2,199.120.110.21,-,-,01/Jul/1995:00:00:09 -0400,GET,/shuttle/missions/sts-73/mission-sts-73.html,HTTP/1.0,200,4085
3,burger.letters.com,-,-,01/Jul/1995:00:00:11 -0400,GET,/shuttle/countdown/liftoff.html,HTTP/1.0,304,0
4,199.120.110.21,-,-,01/Jul/1995:00:00:11 -0400,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,HTTP/1.0,200,4179


In [18]:
# Popular hosts
from pyspark.sql.functions import desc

df_clean.groupBy("host").count().orderBy(desc("count")).limit(10).toPandas()

,host,count
0,piweba3y.prodigy.com,17572
1,piweba4y.prodigy.com,11591
2,piweba1y.prodigy.com,9868
3,alyssa.prodigy.com,7852
4,siltb10.orl.mmc.com,7573
5,piweba2y.prodigy.com,5922
6,edams.ksc.nasa.gov,5434
7,163.206.89.4,4906
8,news.ti.com,4863
9,disarray.demon.co.uk,4353


In [19]:
# Popular content
df_clean.groupBy("endpoint").count().orderBy(desc("count")).limit(10).toPandas()

,endpoint,count
0,/images/NASA-logosmall.gif,111330
1,/images/KSC-logosmall.gif,89638
2,/images/MOSAIC-logosmall.gif,60467
3,/images/USA-logosmall.gif,60013
4,/images/WORLD-logosmall.gif,59488
5,/images/ksclogo-medium.gif,58801
6,/images/launch-logo.gif,40871
7,/shuttle/countdown/,40278
8,/ksc.html,40226
9,/images/ksclogosmall.gif,33585


In [20]:
# Large files
df_clean.createOrReplaceTempView("cleanlog")
spark.sql("""select endpoint, content_size from cleanlog order by content_size desc""").limit(10).toPandas()

,endpoint,content_size
0,/images/cdrom-1-95/img0007.jpg,99981
1,/shuttle/missions/sts-71/movies/sts-71-launch.mpg,999424
2,/shuttle/missions/sts-71/movies/sts-71-launch.mpg,999424
3,/history/apollo/apollo-13/images/index.gif,99942
4,/history/apollo/apollo-13/images/index.gif,99942
5,/history/apollo/apollo-13/images/index.gif,99942
6,/history/apollo/apollo-13/images/index.gif,99942
7,/history/apollo/apollo-13/images/index.gif,99942
8,/history/apollo/apollo-13/images/index.gif,99942
9,/history/apollo/apollo-13/images/index.gif,99942


In [21]:
# Cast content_size column in int
from pyspark.sql.functions import expr
df_cast_type_column = df_clean.withColumn("content_size_bytes", expr("cast(content_size as int)"))
df_cast_type_column.limit(5).toPandas()

,host,client_identd,user_id,date_time,method,endpoint,protocol,response_code,content_size,content_size_bytes
0,199.72.81.55,-,-,01/Jul/1995:00:00:01 -0400,GET,/history/apollo/,HTTP/1.0,200,6245,6245
1,unicomp6.unicomp.net,-,-,01/Jul/1995:00:00:06 -0400,GET,/shuttle/countdown/,HTTP/1.0,200,3985,3985
2,199.120.110.21,-,-,01/Jul/1995:00:00:09 -0400,GET,/shuttle/missions/sts-73/mission-sts-73.html,HTTP/1.0,200,4085,4085
3,burger.letters.com,-,-,01/Jul/1995:00:00:11 -0400,GET,/shuttle/countdown/liftoff.html,HTTP/1.0,304,0,0
4,199.120.110.21,-,-,01/Jul/1995:00:00:11 -0400,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,HTTP/1.0,200,4179,4179


In [23]:
df_cast_type_column.createOrReplaceTempView("cleantypelog")
spark.sql("""select endpoint, content_size from cleantypelog order by content_size_bytes desc""").limit(10).toPandas()

,endpoint,content_size
0,/shuttle/countdown/video/livevideo.jpeg,6823936
1,/statistics/1995/bkup/Mar95_full.html,3155499
2,/statistics/1995/bkup/Mar95_full.html,3155499
3,/statistics/1995/bkup/Mar95_full.html,3155499
4,/statistics/1995/bkup/Mar95_full.html,3155499
5,/statistics/1995/bkup/Mar95_full.html,3155499
6,/statistics/1995/bkup/Mar95_full.html,3155499
7,/statistics/1995/bkup/Mar95_full.html,3155499
8,/statistics/1995/Jun/Jun95_reverse_domains.html,2973350
9,/statistics/1995/Jun/Jun95_reverse_domains.html,2973350
